In [1]:
import re
from tools.naver import NaverReportScraper

기업 리포트

In [ ]:
# NaverReportScraper 인스턴스 생성
scraper = NaverReportScraper()

scraper.scrape_company(start_page=1, end_page=2)
report_list = scraper.get_reports()
scraper.save_to_db()

AttributeError: 'NaverReportScraper' object has no attribute 'scrape'

In [ ]:
print("AI가 요약한 결과입니다. 정보의 정확성은 보장하지 않습니다.")
print()
print("================================================")

# 회사명 기준으로 report_list 정렬
sorted_report_list = sorted(report_list, key=lambda x: x["회사명"], reverse=True)

for report_dict in sorted_report_list:
    print("## ", report_dict["리포트명"], "|", report_dict["회사명"], "|", report_dict["증권사"], "|", report_dict["날짜"], " ##")
    print()
    # 요약을 문장 단위로 쪼개서 '. ' 뒤에 엔터를 넣어 출력
    summary = report_dict["요약"]
    # '. '로 끝나는 문장 뒤에만 줄바꿈 추가 (예: 7.9만원 등은 줄바꿈하지 않음)
    sentences = re.split(r'(?<=\.\s)', summary)
    for sentence in sentences:
        if sentence.strip():  # 빈 문장은 출력하지 않음
            print("◼︎ ", sentence.strip())
    print("================================================")
    print()

산업리포트

In [3]:
scraper = NaverReportScraper()
scraper.scrape_industry()
report_list = scraper.get_reports()
scraper.save_to_db()

산업리포트:   0%|          | 0/49 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P21' is an invalid float value
Cannot set gray non-stroke color because /'P56' is an invalid float value
Cannot set gray non-stroke color because /'P79' is an invalid float value
Cannot set gray non-stroke color because /'P81' is an invalid float value
Cannot set gray non-stroke color because /'P83' is an invalid float value
Cannot set gray non-stroke color because /'P85' is an invalid float value
Cannot set gray non-stroke color because /'P87' is an invalid float value
Cannot set gray non-stroke color because /'P88' is an invalid float value
Cannot set gray non-stroke color because /'P90' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P93' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float

25건의 리포트가 DB에 저장되었습니다.


In [6]:
# AI 요약 결과 출력
print("AI가 요약한 결과입니다. 정보의 정확성은 보장하지 않습니다.")
print()

print("#### 산업리포트 ####")

print("================================================")

# 산업명 기준으로 report_list 정렬
sorted_report_list = sorted(report_list, key=lambda x: x["산업명"], reverse=True)

for report_dict in sorted_report_list:
    print("## ", report_dict["리포트명"], "|", report_dict["산업명"], "|", report_dict["증권사"], "|", report_dict["날짜"], " ##")
    print()
    # 요약을 문장 단위로 쪼개서 '. ' 뒤에 엔터를 넣어 출력
    summary = report_dict["요약"]
    # '. '로 끝나는 문장 뒤에만 줄바꿈 추가 (예: 7.9만원 등은 줄바꿈하지 않음)
    sentences = re.split(r'(?<=\.\s)', summary)
    for sentence in sentences:
        if sentence.strip():  # 빈 문장은 출력하지 않음
            print("◼︎ ", sentence.strip())
    print("================================================")
    print()

AI가 요약한 결과입니다. 정보의 정확성은 보장하지 않습니다.

#### 산업리포트 ####
##  Steel Alive | 철강금속 | 한화투자증권 | 25.07.21  ##

◼︎  2025년 철강 산업은 중국의 공급 축소와 글로벌 무역 규제 강화에 힘입어 구조적 업황 회복 국면에 진입하고 있다.
◼︎  중국 정부의 감산 정책과 시장 내 자발적 생산 조정이 동시에 진행되며, 단기적으로는 연간 4~5% 수준의 실질 감산이 현실화될 가능성이 높다.
◼︎  장기적으로는 설비 치환 정책 등 구조적 개혁을 통해 영구적인 생산능력 감축이 추진될 전망이며, 이는 글로벌 공급 과잉 우려를 완화하는 핵심 동력으로 작용할 것이다.
◼︎  글로벌 보호무역주의 확산과 국내 반덤핑 조치 강화는 중국산 저가 철강재의 유입을 억제해 국내 철강사의 수익성 개선에 기여할 것으로 보인다.
◼︎  중국 부동산 시장의 점진적 회복도 철강 수요 측면에서 긍정적 신호로 작용하며, 2026년 이후 신규 착공 증가가 본격화될 경우 추가적인 업황 반등이 기대된다.
◼︎  산업 내에서는 양적 성장의 한계에 직면함에 따라 고부가가치 제품 중심의 질적 성장 전략이 중요해지고 있다.
◼︎  현대제철의 미국 전기로 투자 등은 무역장벽 대응과 저탄소 생산체제 구축, 북미 고부가 자동차강판 시장 공략 등 중장기 경쟁력 강화의 전략적 포석으로 평가된다.
◼︎  투자 전략 측면에서는 업황 저점 구간에서 분할 매수를 시작하고, 중국 부동산 회복이 실물 지표로 확인되는 시점에 추가 비중 확대가 유효하다.
◼︎  단기 업황 반등에는 현대제철이, 중장기 성장성 측면에서는 고부가 판재와 이차전지 소재 등 비철강 성장 동력을 보유한 POSCO홀딩스가 주목받고 있다.
◼︎  전반적으로 공급 구조 변화와 글로벌 무역 환경의 변화가 철강 산업의 질적 전환과 투자 매력도를 높이고 있다.

##  높아진 중국 철강 구조조정 가능성, 강도가 .. | 철강금속 | 하나증권 | 25.07.21  ##

◼︎  중국 정부가 철강산업의 무질서한